In [2]:
import requests
import json
import xml.etree.ElementTree as ET

def get_osm_map_bounds(osm_map_dir):
    bounds = ET.parse(osm_map_dir).getroot().find("bounds")
    return dict(bounds.attrib.items())

def get_region_issues(osm_map_dir, city_key, labels, out_dir):
    coords = get_osm_map_bounds(osm_map_dir)
    resp = requests.get(
        f"https://sidewalk-{city_key}.cs.washington.edu/v3/api/labelClusters",
        params={ "bbox": "{},{},{},{}".format(coords["minlon"], coords["minlat"], coords["maxlon"], coords["maxlat"]), "labelType": ",".join(labels) },
        timeout=20
    )
    if resp.status_code == 200:
        data = resp.json()
        with open(out_dir, "w") as f:
            json.dump(data, f, indent=4)

get_region_issues("./test_data/small_map.osm", "sea", ["NoCurbRamp", "SurfaceProblem", "NoSidewalk"], "labels.json")

In [3]:
import datetime
import json
import numpy as np
import pandas as pd

EARTH_RAD = 6_378_137

def get_region_info(json_path, num_days=365*5):
    with open(json_path, "r") as f:
        file = json.load(f)
    markers = []
    if "features" in file.keys():
        for f in file["features"]:
            date = f["properties"]["avg_image_capture_date"].split("-")
            if (datetime.datetime.now() - datetime.datetime(int(date[0]), int(date[1]), int(date[2][:2]))).days <= num_days:
                markers.append((f["properties"]["label_type"], f["geometry"]["coordinates"])) 
    return markers

def get_translation_from_center(center_coords, cur_coords):
    center_lng, center_lat = center_coords
    cur_lng, cur_lat = cur_coords
    lat_diff = np.deg2rad(cur_lat - center_lat)
    lng_diff = np.deg2rad(cur_lng - center_lng)
    # local tangent plane approximation
    x = lng_diff * EARTH_RAD * np.cos(np.deg2rad(center_lat))
    y = lat_diff * EARTH_RAD
    return (x, y)

def centered_region_info_to_csv(markers, osm_map_dir, out_path):
    bounds = get_osm_map_bounds(osm_map_dir)
    center_lng, center_lat = (float(bounds["minlon"]) + float(bounds["maxlon"])) / 2, (float(bounds["minlat"]) + float(bounds["maxlat"])) / 2

    df = pd.DataFrame(columns=["issue label", "dx", "dy", "lng", "lat"])
    for m in markers:
        lng, lat = m[1]
        dx, dy = get_translation_from_center((center_lng, center_lat), (lng, lat))
        df.loc[len(df)] = [m[0], dx, dy, lng, lat]
    df.to_csv(out_path)


centered_region_info_to_csv(get_region_info("labels.json", 365*20), "test_data/small_map.osm", "translations.csv")

In [ ]:
import osmnx as ox
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
import math

EARTH_RAD = 6_378_137
walkable_edges = ["footway", "path", "pedestrian", "residential", "living_street", "service"]

def cost(coord1, coord2, lat_center):
    dx = (coord2['x'] - coord1['x']) * math.pi/180 * EARTH_RAD * math.cos(math.radians(lat_center))
    dy = (coord2['y'] - coord1['y']) * math.pi/180 * EARTH_RAD
    return math.sqrt(dx ** 2 + dy ** 2)


# parses OSM file to extract nodes for A* implementation and define adjacency map, filtered to walkable edges
def parse_osm_file(path, marker_csv_path, offset_x, offset_y):
    G = ox.graph_from_xml(filepath=path, simplify=True)

    nonwalkable_edges = []
    for u, v, k, data in G.edges(keys=True, data=True):
        highway = data.get("highway")
        if highway is None or highway not in walkable_edges:
            nonwalkable_edges.append((u, v, k))
    print("num nonwalkable edges: {}".format(len(nonwalkable_edges)))
    G.remove_edges_from(nonwalkable_edges)
    print("num nodes: {}, num edges: {}".format(len(G.nodes()), len(G.edges())))


    # add marker edges
    df = pd.read_csv(marker_csv_path)
    all_lats = [G.nodes[n]['y'] for n in G.nodes]
    lat_center = np.mean(all_lats)

    for i in range(len(df)):
        id = max(G.nodes) + 1
        issue_x, issue_y = df.loc[i, "lng"], df.loc[i, "lat"]

        # find and connect the two nearest nodes to the marker (TODO maybe these cross throguh city blocks)
        coords = np.array([[G.nodes[n]['x'], G.nodes[n]['y']] for n in G.nodes])
        tree = cKDTree(coords)
        _, indices = tree.query([issue_x, issue_y], 2)
        G.add_node(id, x=issue_x, y=issue_y, issue=True)

        nearest_nodes = [list(G.nodes)[index] for index in indices]
        for node in nearest_nodes:
            c = cost(G.nodes[id], G.nodes[node], lat_center)
            G.add_edge(id, node, length=c)
            G.add_edge(node, id, length=c)

    node_locs_from_center = {}
    for node_id, data in G.nodes(data=True):
        x = data.get('x')
        y = data.get('y')
        bounds = get_osm_map_bounds(path)
        center_lng, center_lat = (float(bounds["minlon"]) + float(bounds["maxlon"])) / 2, (float(bounds["minlat"]) + float(bounds["maxlat"])) / 2
        dx, dy = get_translation_from_center((center_lng, center_lat), (x, y))
        dx += offset_x
        dy += offset_y
        print(f"Node {node_id}: x = {dx}, y = {dy}")
        node_locs_from_center[node_id] = (dx, dy)

    adj_map = {}
    for u, v, data in G.edges(data=True):
        length = data.get('length')
        if u not in adj_map:
            adj_map[u] = []
        adj_map[u].append((v, length))

    return G, adj_map, node_locs_from_center
    
G, connections, nodes = parse_osm_file("/Users/irislitiu/Downloads/small_map.osm", "./translations.csv", 47.7, -122)

for node_id, (dx, dy) in nodes.items():
    if G.nodes[node_id].get("issue", False):
        print(f"Marker node {node_id} at dx={round(dx, 2)}, dy={round(dy, 2)}")
    else:
        print(f"Regular node {node_id} at dx={round(dx, 2)}, dy={round(dy, 2)}")


num nodes: 312, num edges: 948
num nodes: 312
Node 12234207233: x = -228.3246384771844, y = -67.89872747422336
Node 12289740803: x = -317.9747552235882, y = -174.89902202460618
Node 12234207236: x = -112.46818065836003, y = -26.198446222988636
Node 12289740806: x = -228.57943275784214, y = -93.61352984740569
Node 12289740810: x = -134.10321259390292, y = -26.944286811318833
Node 12289740811: x = -315.5841853633018, y = -93.33523112112513
Node 12289740813: x = -318.1471160603472, y = -109.61014067466752
Node 12234207240: x = -112.6929991399778, y = -10.580321664892097
Node 12234207255: x = -133.93085175714396, y = 135.8493365241066
Node 12234207254: x = -112.17591663076597, y = 136.12763525117816
Node 12289740826: x = -255.42275953030963, y = -108.8420361879185
Node 12289740827: x = -255.9548299385089, y = -93.65805764424334
Node 12291287068: x = 474.06075751581835, y = -268.30720674927704
Node 12291287074: x = 289.8669750605641, y = -221.2190621439626
Node 12291287075: x = 290.12926329

In [ ]:
import json
import pandas as pd

# G should be None if manual
def adjacency_map_to_json(G, connections, nodes, out_path, is_manual=False, offset_x=47.7, offset_y=-122):
    nodes_json = {}
    if is_manual:
        for node_id, (dx, dy, is_issue) in nodes.items():
            nodes_json[node_id] = {"x": dx + offset_x, "y": dy + offset_y, "issue": is_issue}
    else:
        for node_id, (dx, dy) in nodes.items():
            nodes_json[node_id] = {
                "x": dx,
                "y": dy,
                "issue": G.nodes[node_id].get("issue", False)
            }

    adj_json = {}
    for start_id, neighbors in connections.items():
        adj_json[start_id] = [{"to": end_id, "cost": length} for end_id, length in neighbors]
    graph_json = {
        "nodes": nodes_json,
        "adjacency": adj_json
    }

    with open(out_path, "w") as f:
        json.dump(graph_json, f, indent=2)

def adjacency_map_nodes_to_csv(G, nodes, out_path, is_manual=False):
    df = pd.DataFrame(columns=["node id", "x", "y", "issue"])
    if is_manual:
        for node_id, (dx, dy, is_issue) in nodes.items():
            df.loc[len(df)] = [node_id, dx, dy, is_issue]
    else:
        for node_id, (dx, dy) in nodes.items():
            df.loc[len(df)] = [node_id, dx, dy, G.nodes[node_id].get("issue", False)]
    df.to_csv(out_path)
    
nodes = {"1": (10, 0, False), "2": (7, 5, True), "3": (5, 7, False), "4": (-2, -2, False), "5": (-5, -8, False)}
connections = {"1": [(2, 5), (4, 3)], 2: [(3, 8), (5, 4)]}
#adjacency_map_to_json(None, connections, nodes, "adjacency_map_manual.json", is_manual=True)
adjacency_map_nodes_to_csv(None, nodes, "adjacency_map_nodes_manual.csv", is_manual=True)